In [1]:
import pandas as pd
import os
import time

In [2]:
# # Mount Google Drive
# from google.colab import drive

# drive.mount('/content/drive', force_remount=True)

# # Path to the kaggle.json file
# kaggle_json_path = '/content/drive/My Drive/api_keys/kaggle.json'

# # Move kaggle.json into the correct location
# !mkdir -p ~/.kaggle
# !cp '{kaggle_json_path}' ~/.kaggle/

# drive.flush_and_unmount()

In [ ]:
# Move kaggle.json into the correct location
kaggle_json_path = '/content/apis_keys/kaggle.json'
!mkdir -p ~/.kaggle
!cp '{kaggle_json_path}' ~/.kaggle/

In [3]:
!ls -a ~/.kaggle

.  ..  kaggle.json


In [4]:
# Change permissions of kaggle.json to avoid a warning on Kaggle tool startup
!chmod 600 /root/.kaggle/kaggle.json
!ls -a -l ~/.kaggle

total 16
drwxr-xr-x 2 root root 4096 Mar 17 08:47 .
drwx------ 1 root root 4096 Mar 17 08:46 ..
-rw------- 1 root root   68 Mar 17 13:22 kaggle.json


In [5]:
from kaggle.api.kaggle_api_extended import KaggleApi

In [6]:
# Initialize Kaggle API
api = KaggleApi()
# Authenticate with your Kaggle credentials (you only need to do this once)
# This will use your `kaggle.json` file for authentication
api.authenticate()

In [7]:
# Specify the dataset you want to load
dataset_name = 'new-york-city/nyc-parking-tickets'

In [8]:
# Get dataset files
files = api.dataset_list_files(dataset_name)
type(files)

kaggle.models.kaggle_models_extended.ListFilesResult

In [9]:
# prompt: files seem to be empty, which is not expected, it was expected to have in the folder the 4 .csv NYC traffic files
expected_files = [
    "Parking_Violations_Issued_-_Fiscal_Year_2017.csv",
    "Parking_Violations_Issued_-_Fiscal_Year_2015.csv",
    "Parking_Violations_Issued_-_Fiscal_Year_2016.csv",
    "Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_.csv"
]

# Check if all expected files are already present
files_present = all(os.path.exists(file) for file in expected_files)

# If any file is missing, download the dataset
if not files_present:
    api.dataset_download_files(dataset_name, path='./', unzip=True)
else:
    print("All files are already present in the directory.")

# List the files in the current directory
!ls -l


All files are already present in the directory.
total 8761704
drwxr-xr-x 2 root root       4096 Mar 17 08:40 apis_keys
-rw-r--r-- 1 root root 1869025315 Mar 17 08:53 Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_.csv
-rw-r--r-- 1 root root 2864071408 Mar 17 08:53 Parking_Violations_Issued_-_Fiscal_Year_2015.csv
-rw-r--r-- 1 root root 2151937808 Mar 17 08:54 Parking_Violations_Issued_-_Fiscal_Year_2016.csv
-rw-r--r-- 1 root root 2086913576 Mar 17 08:54 Parking_Violations_Issued_-_Fiscal_Year_2017.csv
drwxr-xr-x 1 root root       4096 Mar 14 13:27 sample_data


In [10]:
# Check if the files are now present
files_result = api.dataset_list_files(dataset_name)
file_names = files_result.files
file_names

[Parking_Violations_Issued_-_Fiscal_Year_2017.csv,
 Parking_Violations_Issued_-_Fiscal_Year_2015.csv,
 Parking_Violations_Issued_-_Fiscal_Year_2016.csv,
 Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_.csv]

In [11]:
# Load data for fiscal year 2017 in chunks
file_name = str(file_names[0])
chunk_size = 100000  # adjust as needed
chunks = pd.read_csv(file_name, chunksize=chunk_size, low_memory=False)

In [12]:
# Initialize an empty list to store DataFrames
dfs = []

# Process each chunk
for i, chunk in enumerate(chunks):
    if (i+1) % 10 == 0:
        print(f"Processing chunk {i+1} ...")
    # Process the chunk here, such as performing computations or analysis
    dfs.append(chunk[['Summons Number', 'Vehicle Make', 'Issue Date']])  # Append the processed chunk DataFrame to the list

# Concatenate all DataFrames in the list into a single DataFrame
df = pd.concat(dfs, ignore_index=True)

Processing chunk 10 ...
Processing chunk 20 ...
Processing chunk 30 ...
Processing chunk 40 ...
Processing chunk 50 ...
Processing chunk 60 ...
Processing chunk 70 ...
Processing chunk 80 ...
Processing chunk 90 ...
Processing chunk 100 ...


In [13]:
df.shape

(10803028, 3)

In [14]:
df.head()

,Summons Number,Vehicle Make,Issue Date
0,5092469481,TOYOT,07/10/2016
1,5092451658,TOYOT,07/08/2016
2,4006265037,FORD,08/23/2016
3,8478629828,MITSU,06/14/2017
4,7868300310,INTER,11/21/2016


In [15]:
df.tail()

,Summons Number,Vehicle Make,Issue Date
10803023,1415891400,HONDA,11/02/2068
10803024,1384716543,TOYOT,07/12/2069
10803025,1413536554,NaN,08/14/2069
10803026,1415514203,JEEP,11/15/2069
10803027,1415995370,TOYOT,11/19/2069


In [16]:
!pip install psutil

In [17]:
# Get the initial memory usage
import psutil
initial_memory = psutil.virtual_memory().used

# Delete the list of DataFrames
del dfs

# Iterate over each chunk DataFrame and delete it
for chunk in chunks:
    del chunk

# Optionally, you can also collect garbage explicitly
import gc
gc.collect()

# Get the final memory usage
final_memory = psutil.virtual_memory().used

# Display the initial and final memory usage
print(f"Initial memory usage: {initial_memory / (1024 * 1024)} MB")
print(f"Final memory usage: {final_memory / (1024 * 1024)} MB")

Initial memory usage: 1215.05859375 MB
Final memory usage: 1217.953125 MB


# **The test dataset**
In this case, a medium-large dataset for the process would be good enough to show the differences. The NYC Parking Tickets are a good dataset for this evaluation. It has 42.3M rows from Aug 2013-June 2017 with 51 columns including Registration State, Vehicle Make, and Vehicle Color that are interesting to know the insights. We will use the fiscal 2017 dataset with 10.8M rows, and the file size is about 2.09G.

# **The evaluation process**
Due to the entire running time that includes reading the data into memory, it is necessary to consider the data loading separately.
We’d process the same call 5x times to avoid edge cases and use the median value to report as our final performance result.

In [18]:
# Helper function to repeat and compute the median
from itertools import repeat
from statistics import median
import functools
import time

## repeat a given function multiple times, append the execution duration to a list
def record_timer(func, times = 5):
    durations = []
    for _ in repeat(None, times):
        start_time = time.perf_counter()
        value = func()
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f"Finished {func.__name__!r} in {run_time:.10f} secs")
        durations.append(run_time)
    return value, durations
## Decorator and compute the median of the function
def repeat_executor(times=5):
    def timer(func):
        """Print the runtime of the decorated function"""
        @functools.wraps(func)
        def wrapper_timer(*args, **kwargs):
            value, durations = record_timer(func, times=times)
            print(f'{median(list(durations))}')
            return value
        return wrapper_timer
    return timer

# **Pandas: The Baseline**
To set up the baseline for comparison, we shall examine the famous use cases for daily analytics jobs: filter, aggregation, joins, and window function.

**filter:** find the Vehicle Make is BMW
**aggregation:** group by Vehicle Make and perform count
**join:** SELF join on Summons Number
**window function:** rank the Vehicle Make based on the count of the

I selected on only the used fields for our testing, which are ‘Summons Number’, ‘Vehicle Make’, ‘Issue Date’ .
Note if I choose to select everything, the last two queries run significantly slower.

In [19]:
# ## Filter on the Vehicle Make for BMW
@repeat_executor(times=5)
def test_filter():
    return df[df['Vehicle Make'] == 'BMW']['Summons Number']

# # ## Group By on the Vehicle Make and Count
@repeat_executor(times=5)
def test_groupby():
    return df.groupby("Vehicle Make").agg({"Summons Number":'count'})

# # ## SELF join
@repeat_executor(times=5)
def test_self_join():
    return df.set_index("Summons Number").join(df.set_index("Summons Number"), how="inner", rsuffix='_other').reset_index()['Summons Number']

## window function
@repeat_executor(times=5)
def test_window_function():
    df['summon_rank'] = df.sort_values("Issue Date",ascending=False) \
        .groupby("Vehicle Make") \
        .cumcount() + 1
    return df


In [20]:
test_filter()
# # The median time is 1.337s

Finished 'test_filter' in 1.5178547170 secs
Finished 'test_filter' in 1.3595423910 secs
Finished 'test_filter' in 1.3368903400 secs
Finished 'test_filter' in 1.3332162510 secs
Finished 'test_filter' in 1.3194181990 secs
1.3368903400005365


7           4628525523
9           4006478550
19          4630524241
41          4629806404
78          8581153938
               ...    
10802832    1414756264
10802856    1334903219
10802897    1422581445
10802899    1373855903
10802968    1388754885
Name: Summons Number, Length: 374929, dtype: int64

In [21]:
test_groupby()
# # The median time is 1.816s

Finished 'test_groupby' in 1.8360642820 secs
Finished 'test_groupby' in 1.8424003100 secs
Finished 'test_groupby' in 1.8133889580 secs
Finished 'test_groupby' in 1.8159573680 secs
Finished 'test_groupby' in 1.5324741210 secs
1.8159573680022731


,Summons Number
Vehicle Make,
",FREI",2
",LEXU",1
",MACK",1
.CHRY,1
.INFI,2
...,...
]FREI,1
`,3
`APRI,1


In [22]:
test_self_join()
# # The median time is 10.669s

Finished 'test_self_join' in 13.6588711400 secs
Finished 'test_self_join' in 10.6688420530 secs
Finished 'test_self_join' in 10.6825620670 secs
Finished 'test_self_join' in 10.2966491250 secs
Finished 'test_self_join' in 10.4372144370 secs
10.668842053000844


0           5092469481
1           5092451658
2           4006265037
3           8478629828
4           7868300310
               ...    
10803023    1415891400
10803024    1384716543
10803025    1413536554
10803026    1415514203
10803027    1415995370
Name: Summons Number, Length: 10803028, dtype: int64

In [23]:
test_window_function()
# # The median time is 40.861s

Finished 'test_window_function' in 43.5653343580 secs
Finished 'test_window_function' in 41.9792342640 secs
Finished 'test_window_function' in 39.3702585710 secs
Finished 'test_window_function' in 40.8608218110 secs
Finished 'test_window_function' in 40.0871344960 secs
40.86082181099846


,Summons Number,Vehicle Make,Issue Date,summon_rank
0,5092469481,TOYOT,07/10/2016,554380.0
1,5092451658,TOYOT,07/08/2016,557463.0
2,4006265037,FORD,08/23/2016,456995.0
3,8478629828,MITSU,06/14/2017,52384.0
4,7868300310,INTER,11/21/2016,17974.0
...,...,...,...,...
10803023,1415891400,HONDA,11/02/2068,168584.0
10803024,1384716543,TOYOT,07/12/2069,547609.0
10803025,1413536554,NaN,08/14/2069,NaN
10803026,1415514203,JEEP,11/15/2069,41296.0


# **DuckDb: Efficient OLAP In-Process DB**
DuckDB is gaining popularity as its columnar-vectorized engine powers analytical types of queries. It’s an analytical or OLAP version of SQLite, a widely adopted simple embedded in-process DBMS.

Although it’s a DBMS, installation isn’t complex compared to Microsoft SQL Server or Postgres; Additionally, no external dependencies are required to run a query. I am astonished how easy it is to execute a SQL query with DuckDb CLI.

If you prefer SQL interface, DuckDb might be your best alternative to performing data analysis directly on CSV or Parquet file. Let’s continue with some code examples and simultaneously show how straightforward it is to work with SQL with DuckDb.

DuckDb has a magic read_csv_auto function to infer a CSV file and load that data into memory. At runtime, I found I have to change SAMPLE_SIZE=-1 to skip sampling due some fields in my dataset isn’t inferred correctly, with sampling is default as 1,000 rows.

In [24]:
import duckdb
con = duckdb.connect(database=':memory:')
query = f"""CREATE TABLE parking_violations
    AS SELECT "Summons Number", "Vehicle Make", "Issue Date"
    FROM read_csv_auto('{file_name}', delim=',', SAMPLE_SIZE=-1);"""
con.execute(query)
con.execute("""SELECT COUNT(1) FROM parking_violations""")
print(con.fetchall())

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[(10803028,)]


In [25]:
# ## Filter on the Vehicle Make for BMW
@repeat_executor(times=5)
def test_filter():
    con.execute("""
        SELECT * FROM parking_violations WHERE "Vehicle Make" = 'BMW'
        """)
    return con.fetchall()
# # ## Group By on the Vehicle Make and Count
@repeat_executor(times=5)
def test_groupby():
    con.execute("""
        SELECT COUNT("Summons Number") FROM parking_violations GROUP BY "Vehicle Make"
        """)
    return con.fetchall()
# # # ## SELF join
@repeat_executor(times=5)
def test_self_join():
    con.execute("""
        SELECT a."Summons Number"
        FROM parking_violations a
        INNER JOIN parking_violations b on a."Summons Number" = b."Summons Number"
        """)
    return con.fetchall()
# ## window function
@repeat_executor(times=5)
def test_window_function():
    con.execute("""
        SELECT *, ROW_NUMBER() OVER (PARTITION BY "Vehicle Make" ORDER BY "Issue Date")
        FROM parking_violations
        """)
    return con.fetchall()

In [26]:
test_filter()
# The median time is 0.608s

Finished 'test_filter' in 0.6265138910 secs
Finished 'test_filter' in 0.6244227160 secs
Finished 'test_filter' in 0.6081501870 secs
Finished 'test_filter' in 0.5969299110 secs
Finished 'test_filter' in 0.5797852330 secs
0.6081501869994099


[(4628525523, 'BMW', datetime.date(2016, 12, 21)),
 (4006478550, 'BMW', datetime.date(2016, 10, 5)),
 (4630524241, 'BMW', datetime.date(2017, 2, 3)),
 (4629806404, 'BMW', datetime.date(2017, 1, 23)),
 (8581153938, 'BMW', datetime.date(2017, 6, 14)),
 (7128799357, 'BMW', datetime.date(2016, 9, 29)),
 (8491819083, 'BMW', datetime.date(2017, 3, 3)),
 (4627653700, 'BMW', datetime.date(2016, 12, 1)),
 (4634734539, 'BMW', datetime.date(2017, 6, 5)),
 (7752393691, 'BMW', datetime.date(2016, 12, 3)),
 (1412956201, 'BMW', datetime.date(2016, 7, 20)),
 (7970393330, 'BMW', datetime.date(2016, 8, 17)),
 (8506469132, 'BMW', datetime.date(2017, 4, 5)),
 (8527214490, 'BMW', datetime.date(2017, 5, 16)),
 (7036998684, 'BMW', datetime.date(2017, 2, 21)),
 (8505255379, 'BMW', datetime.date(2017, 1, 27)),
 (8421031235, 'BMW', datetime.date(2016, 10, 18)),
 (8318510150, 'BMW', datetime.date(2016, 8, 2)),
 (7725863710, 'BMW', datetime.date(2016, 9, 26)),
 (8537965042, 'BMW', datetime.date(2017, 5, 16)),
 (8

In [27]:
test_groupby()
# # The median time is 0.669s

Finished 'test_groupby' in 0.7378878790 secs
Finished 'test_groupby' in 0.6692460730 secs
Finished 'test_groupby' in 0.6374079410 secs
Finished 'test_groupby' in 0.6611286840 secs
Finished 'test_groupby' in 0.7163064900 secs
0.669246073000977


[(1211451,),
 (1280958,),
 (243763,),
 (374929,),
 (429158,),
 (176559,),
 (78174,),
 (62793,),
 (1079238,),
 (255112,),
 (135732,),
 (348145,),
 (177227,),
 (85863,),
 (108974,),
 (5524,),
 (107291,),
 (574,),
 (140760,),
 (33531,),
 (72,),
 (107809,),
 (49229,),
 (39172,),
 (5182,),
 (4007,),
 (73047,),
 (37,),
 (13984,),
 (6152,),
 (7948,),
 (12145,),
 (1,),
 (10091,),
 (2,),
 (13605,),
 (3485,),
 (1428,),
 (43654,),
 (1324,),
 (4030,),
 (24437,),
 (6031,),
 (1329,),
 (5179,),
 (4115,),
 (1815,),
 (65,),
 (58,),
 (1342,),
 (225,),
 (5468,),
 (15,),
 (3270,),
 (33,),
 (1,),
 (4814,),
 (58,),
 (939,),
 (284,),
 (1,),
 (919,),
 (532,),
 (1,),
 (3818,),
 (301,),
 (102,),
 (5,),
 (1403,),
 (116,),
 (1266,),
 (1,),
 (133,),
 (114,),
 (1373,),
 (1700,),
 (526,),
 (344,),
 (336,),
 (1747,),
 (4,),
 (1359,),
 (68,),
 (1616,),
 (46,),
 (194,),
 (3,),
 (74,),
 (229,),
 (142,),
 (309,),
 (117,),
 (1741,),
 (280,),
 (5,),
 (390,),
 (535,),
 (6,),
 (6,),
 (797,),
 (2,),
 (47,),
 (20,),
 (80,),
 (

In [28]:
test_self_join()
# # The median time is 10.753s

Finished 'test_self_join' in 11.5469365060 secs
Finished 'test_self_join' in 11.3726329020 secs
Finished 'test_self_join' in 10.0121294600 secs
Finished 'test_self_join' in 10.7529721920 secs
Finished 'test_self_join' in 9.0852537670 secs
10.752972192000016


[(5092469481,),
 (5092451658,),
 (4006265037,),
 (7868300310,),
 (5096917368,),
 (1413609545,),
 (4628525523,),
 (4627113330,),
 (4006478550,),
 (1407740258,),
 (8009901763,),
 (4625926610,),
 (1416492320,),
 (1413656420,),
 (7959486440,),
 (5093620865,),
 (1416638830,),
 (4630524241,),
 (8505131836,),
 (8513520615,),
 (8556155431,),
 (8513914599,),
 (8344027755,),
 (8040135217,),
 (8466021980,),
 (7047487591,),
 (8544889610,),
 (4628712840,),
 (8162054819,),
 (8506053195,),
 (4628491689,),
 (8534116519,),
 (8479417420,),
 (4629806404,),
 (8522218316,),
 (8482374059,),
 (8523051284,),
 (4628303630,),
 (8431017041,),
 (8263642668,),
 (8515215408,),
 (1416527722,),
 (7411972721,),
 (8532505170,),
 (8558750191,),
 (8486910511,),
 (7536344478,),
 (8446524806,),
 (8063538547,),
 (7081935680,),
 (8410006650,),
 (7808333074,),
 (8034466670,),
 (7059132030,),
 (8518670070,),
 (8335524634,),
 (8558610137,),
 (8358531096,),
 (7249114220,),
 (8564313893,),
 (1418809688,),
 (8552758799,),
 (141614

In [29]:
test_window_function()
# # The median time is 25.546s

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Finished 'test_window_function' in 22.5394501440 secs


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Finished 'test_window_function' in 25.5372804500 secs


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Finished 'test_window_function' in 25.7919131610 secs


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Finished 'test_window_function' in 23.5352673670 secs
24.545720991001872


[(1386078797, 'AIBJ', datetime.date(2016, 10, 7), 1),
 (1402887048, 'AWD', datetime.date(2016, 12, 1), 1),
 (1404019418, 'BI/BI', datetime.date(2016, 12, 20), 1),
 (1413620152, 'BI/BI', datetime.date(2016, 12, 28), 2),
 (1417624875, 'BK', datetime.date(2017, 4, 21), 1),
 (1411737921, 'BL/BD', datetime.date(2016, 8, 13), 1),
 (5094459570, 'BL/BD', datetime.date(2016, 11, 23), 2),
 (4629642341, 'BL/BD', datetime.date(2017, 1, 17), 3),
 (5095518119, 'BL/BD', datetime.date(2017, 2, 28), 4),
 (1408875834, 'BL/BD', datetime.date(2017, 4, 9), 5),
 (1415575460, 'BRUCK', datetime.date(2016, 9, 29), 1),
 (1414873724, 'BRUCK', datetime.date(2016, 11, 26), 2),
 (1407341297, 'CANAM', datetime.date(2016, 7, 20), 1),
 (5092912716, 'CANAM', datetime.date(2016, 8, 8), 2),
 (5092960863, 'CANAM', datetime.date(2016, 8, 11), 3),
 (4623648291, 'CANAM', datetime.date(2016, 9, 20), 4),
 (5093949670, 'CANAM', datetime.date(2016, 10, 18), 5),
 (1402578933, 'CANAM', datetime.date(2016, 11, 14), 6),
 (4634261133

# **Polars: Astonishing Fast Build On Rust + Arrow**
Polars was created by Ritchie Vink. Ritchie also has a blog post, “I wrote one of the fastest DataFrame libraries,” and it was well-received. The impressive part for Polars is that on the Database-like ops benchmark by h2oai, it ranked the top on the group by and join operations.

Here are a few reasons Polars can replace Pandas:

Polars starts with the parallelization of DataFrame from the beginning. It doesn’t restrict itself to single-core operation.
PyPolars is Rust-based with Python bindings, which has outstanding performance comparable to C, and “Arrow Columnar Format” is an excellent choice for the analytics OLAP type query.
Lazy evaluation: plan (not execute) the query until triggered. This can be used to optimize queries like additional pushdown further.

In [37]:
import polars as pl

# Load data for fiscal year 2017
pl_df = pl.read_csv(file_name, columns=["Summons Number", "Vehicle Make", "Issue Date"])

In [39]:
pl_df.shape, type(pl_df)

((10803028, 3), polars.dataframe.frame.DataFrame)

In [40]:
pl_df.head()

Summons Number,Issue Date,Vehicle Make
i64,str,str
5092469481,"""07/10/2016""","""TOYOT"""
5092451658,"""07/08/2016""","""TOYOT"""
4006265037,"""08/23/2016""","""FORD"""
8478629828,"""06/14/2017""","""MITSU"""
7868300310,"""11/21/2016""","""INTER"""


In [41]:
pl_df.tail()

Summons Number,Issue Date,Vehicle Make
i64,str,str
1415891400,"""11/02/2068""","""HONDA"""
1384716543,"""07/12/2069""","""TOYOT"""
1413536554,"""08/14/2069""",null
1415514203,"""11/15/2069""","""JEEP"""
1415995370,"""11/19/2069""","""TOYOT"""


In [56]:
# ## Filter on the Vehicle Make for BMW
@repeat_executor(times=5)
def test_filter():
    return pl_df.filter(pl.col('Vehicle Make') == 'BMW').select('Summons Number')

# # ## Group By on the Vehicle Make and Count
@repeat_executor(times=5)
def test_groupby():
    return pl_df.groupby("Vehicle Make").agg(pl.col("Summons Number").count())

# # # ## SELF join
@repeat_executor(times=5)
def test_self_join():
    return pl_df.join(pl_df, on="Summons Number", how="inner").select('Summons Number')

# ## window function
@repeat_executor(times=5)
def test_window_function():
    return pl_df.select([
        'Summons Number',
        'Vehicle Make',
        'Issue Date',
        pl.col('Issue Date').sort().reverse().cum_count().over("Vehicle Make").alias("summon_rank")
    ])

In [43]:
test_filter()
# # The median time is 0.251s

Finished 'test_filter' in 0.4674884030 secs
Finished 'test_filter' in 0.2513973070 secs
Finished 'test_filter' in 0.2527592850 secs
Finished 'test_filter' in 0.2150037820 secs
Finished 'test_filter' in 0.2487210480 secs
0.2513973069981148


Summons Number
i64
4628525523
4006478550
4630524241
4629806404
8581153938
7128799357
8491819083
4627653700
4634734539


In [45]:
test_groupby()
# # # The median time is 0.937s

<ipython-input-42-d2333c73a13c>:9: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  return pl_df.groupby("Vehicle Make").agg(pl.col("Summons Number").count())


Finished 'test_groupby' in 0.9885797960 secs
Finished 'test_groupby' in 0.9856212970 secs
Finished 'test_groupby' in 0.9365687920 secs
Finished 'test_groupby' in 0.8802085340 secs
Finished 'test_groupby' in 0.9111573690 secs
0.9365687919998891


Vehicle Make,Summons Number
str,u32
"""LRNW""",1
"""IND""",2
"""HUMEM""",1
"""JARGU""",1
"""M ACK""",1
"""RETUB""",1
"""TUSCA""",1
"""GRATE""",1
"""MUS""",1


In [49]:
test_self_join()
# # # The median time is 8.911s

Finished 'test_self_join' in 9.2196028490 secs
Finished 'test_self_join' in 8.9112545030 secs
Finished 'test_self_join' in 9.8127784900 secs
Finished 'test_self_join' in 8.6525925330 secs
Finished 'test_self_join' in 7.3223702040 secs
8.911254503000237


Summons Number
i64
5092469481
5092451658
4006265037
8478629828
7868300310
5096917368
1413609545
4628525523
4627113330


In [57]:
test_window_function()
# # The median time is 14.710s

Finished 'test_window_function' in 14.7095212100 secs
Finished 'test_window_function' in 14.8085572550 secs
Finished 'test_window_function' in 15.7474084940 secs
Finished 'test_window_function' in 14.0767597550 secs
Finished 'test_window_function' in 13.9912236260 secs
14.709521209999366


Summons Number,Vehicle Make,Issue Date,summon_rank
i64,str,str,u32
5092469481,"""TOYOT""","""07/10/2016""",0
5092451658,"""TOYOT""","""07/08/2016""",1
4006265037,"""FORD""","""08/23/2016""",0
8478629828,"""MITSU""","""06/14/2017""",0
7868300310,"""INTER""","""11/21/2016""",0
5096917368,"""ME/BE""","""06/13/2017""",0
1413609545,"""TOYOT""","""08/03/2016""",2
4628525523,"""BMW""","""12/21/2016""",0
4627113330,"""DODGE""","""11/21/2016""",0
